# Lab 1: Numbers

It seems glaringly obvious that numbers are the key element of computational physics. However, physics describes the world by building a hierarchy of *mathematical models* of reality. These models rely often on real numbers, roughly speaking points along an infinite number line:

![Number line](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Real_number_line.svg/1000px-Real_number_line.svg.png)

Now, of course, as we have discussed, there are numbers, like $\pi$ or $1/3$ which have an infinite number of decimal digits. We still need to calculate with these numbers, but we cannot fit them in our finite computers. This leads to one of the two main sources of error in computations. 

### Exercise
With a partner, discuss the difference between $\pi$ and $1/3$, both of which have an infinite number of digits. What is the *key* difference between their decimal expansions?


## Motivation
Before diving in to learning about number systems and computational representations thereof, let's motivate **why you should care** about it.

### Exercise
In the cell below, write a function `counter` that takes two arguments, `incr` and `n`. The function should use a `for` loop to add `incr` to zero `n` times. **Don't use multiplication!**

In [1]:
def counter(incr, n):
    #n must be a whole number 
    num_to_incr = 0
    for i in range(n):
        num_to_incr += incr
    return num_to_incr

Now, test your function by setting `incr = 2` and `n=100` and subtracting the analytic answer (that is, calculate it by hand). You should get zero.

In [2]:
analytic_answer = 200 #=2*100
test = counter(2,100) - analytic_answer # should be zero
print(test)

0


Now do the same thing again, but set `incr = 0.1` and `n=100`. Now, rather than using the analytic answer, use multiplication: `incr*n`. I've provided test code below.

In [3]:
analytic_answer = 10 #=.1*100
test = counter(.1,100) - analytic_answer # should be zero
print(test) 

-1.9539925233402755e-14


incr = 0.1
n = 100
mult_ans = #fill this in
test = counter(incr,n) - mult_ans
print(test)

**Something weird should have happend!**

In order to understand what is going on, try the same thing but with `incr = 0.2, 0.3, 0.4, 0.5`. What happens with each case? Can you figure out a pattern? 

This weirdness is why we need to spend time understanding how numbers work.

#### Programming tip
Try to write the cell above such that you only need to change a single thing to run the different cases: the value of `incr`. Everything else should just *work*. If you want to get fancy, you could even write another function `test_counter` (or whatever you want to call it) that does everything all at once. 

In [4]:
def test_counter(increases, n):
    """ increases should be a list of all the different values of we want to try increasing by """
    for incr in increases:
        analytical_answer = incr * n
        test = counter(incr,n) - analytical_answer # should be zero
        print(f"The difference between the value we got from the counter function and the " + \
              f"analytically calculated expected value is {test}.")

# Using the test_counter function
increases = [.2, .3, .4, .5]
n = 100
test_counter(increases, n)

The difference between the value we got from the counter function and the analytically calculated expected value is -3.907985046680551e-14.
The difference between the value we got from the counter function and the analytically calculated expected value is 4.973799150320701e-14.
The difference between the value we got from the counter function and the analytically calculated expected value is -7.815970093361102e-14.
The difference between the value we got from the counter function and the analytically calculated expected value is 0.0.


## Binary 

Virtually every computer uses binary to store numbers. A binary number system uses only two values, canonically 0 and 1, in each digit, as opposed to the ten we use in decimal. Very briefly, here is a short table showing the conversion of the first ten integers from decimal to binary:

|Decimal|Binary|
|-------|------|
|00|0000|
|01|0001|
|02|0010|
|03|0011|
|04|0100|
|05|0101|
|06|0110|
|07|0111|
|08|1000|
|09|1001|
|10|1010|

Each digit of the binary number is called a "bit" ("binary digit"). Thus, the bit is the smallest unit of computer memory. One byte is 8 bits. 

Just as an arbitrary number can be expanded in decimal as 

$$x = \sum_{i=-\infty}^{\infty} \alpha_i 10^{i},$$

an arbitrary number can be expanded in binary as 

$$x = \sum_{i=-\infty}^{\infty} \beta_i 2^{i},$$

where $\alpha_i \ne \beta_i$, and obviously most of the coefficents are zero.

`numpy` provides a function (`np.binary_repr()`) to convert *integers* into their binary represenation. Let's practice a bit by using it. Try changing $n$ below. Try a bunch of numbers to get a feel for how binary works. You might try putting `np.binary_repr()` into a loop.

In [5]:
import numpy as np

In [6]:
n = 649303

np.binary_repr(n)

'10011110100001010111'

### Exercise
Create a new code cell below, and see what happens when you try to give a floating point number (e.g. 43.2) to `np.binary_repr()`. Read carefully the output. 

In [7]:
n = 9.2

np.binary_repr(n)

TypeError: 'float' object cannot be interpreted as an integer

## Floating Point Numbers

Throughout this class, we will need to work with computer representations of real numbers: we'll be adding, subtracting, dividing, and multiplying them in order to solve complex problems regarding the world. In order to understand everything in the physical universe, we need to cover length scales ranging from the size of the proton, $l_p \sim 10^{-14}\ \mathrm{m}$,\* to the size of the observable universe, $L_u \sim 10^{27}\ \mathrm{m}$. Taking the ratio of $L_u/l_p \simeq 10^{41}$, we see there is a factor of $10^{41}$ between the smallest and largest length scales.

In order to cover quantities over such a large range of scales, we typically use scientific notation in our paper-and-pencil work, and computers do very much the same thing. Just as in your physics lab courses, you got used to keeping track of only a fixed number of digits of precision. Here, the computer does exactly the same thing. The only tricky part is that the number, which we like to represent in decmial form, is stored in binary. 

Let's analyze scientific notation, taking a typical value, say the mass of the electron $ m_e = 9.10938356 \times 10^{-31} \mathrm{kg}$. 

We can write this schematically as 

$$ m_e = (-1)^s\ m\ \times 10^e,$$

where $s=0$ is called the sign, $m= 9.10938356$ the mantissa or significant, and $e=-31$ the exponent. 

Of course, the computer stores the number $m_e$ in binary, so it must be converted. We'll use the notation $N_{10}$ to mean a number $N$ in base 10, and $N_{2}$ to mean a number in base 2. For example $10_{10} = 1010_{2}$. 

## IEEE 754

There are many ways of storing floating point numbers on a computer; we will only describe one: IEEE 754. This is the most widely used standard for representing these numbers. An IEEE 754 number is represented in the following way:

$$ x = (-1)^s\ 1.f\ \times 2^{e-B},$$

where $s$ is the sign, $f$ is the *fractional part* of the mantissa (note that there is 1 in front), $e$ is the exponent of the number you want to represent, and $B$ is called the *bias*. Note that the total exponent stored in memory $e$  has no sign bit and is thus **always positive**. This explains the need for the bias: if we want to represent actual exponents $p = e -B < 0$, then $B>0$. A floating point number standard that can't hold the mass of an electron is not very useful to scientists. 

IEEE 754 defines two different kinds of floating point numbers: single and double precision. Single precision floating point numbers are called `float`s and take up 32 bits of memory, and double precision is called `double` and take up 64 bits of memory. **Unfortunately, python doesn't respect this naming convention**, chosing instead to use `float` to mean a 64-bit number, and `float32` to mean a 32-bit number. This is only mildly annoying in practice, since you'll probably never need a 32-bit float after this week. 

**We'll use 32-bit single precision as our example here, even though you will almost always work in double precision.** A 32 bit `float` uses 1 bit to hold the sign $s$, 8 bits to hold the exponent $e$, and the remaining 23 bits to hold the mantissa. The bias $B = 127$ for single precision.

Let's consider the exponent first. With 8 bits, the largest value that can be held is with all 8 digits set to 1, 11111111:

In [8]:
# convert a binary digit written as a string (that is, in quotes) prefaced with '0b' to decimal integers
int('0b11111111',base=2) 

255

obviously, the smallest value is 00000000:

In [9]:
int('0b00000000',base=2) 

0

So, $0 \leq e \leq 255$. In IEEE 754, $e=0$ and $e=255$ are special, so normal numbers are restricted to $0 \lt e \lt 255$. 

The fractional part of the mantissa is 23 bits wide, plus the 1 we assue leads the mantissa, for a total of 24 digits. So the largest possible mantissa is (remember, this is really a binary *fraction*: 1.11111111111111111111111)

In [11]:
int('0b111111111111111111111111',base=2)*2**-23

1.9999998807907104

So, putting these together, the largest digit we can store is 

In [12]:
m = int('0b111111111111111111111111',base=2)*2**-23
bias = 127
exp = 2**(254-bias)

print("{:20.15e}".format(m*exp))

3.402823466385289e+38


As mentioned above, the case $e = 0$ is a special one. In this case, the 23 mantissa bits represent the entire mantissa, and the leading digit is zero (instead of one, as is usual). Also, the total stored exponent will be -126. So, the smallest number is thus

In [13]:
m = int('0b000000000000000000000001',base=2)*2**-23
exp = 2**(-126)

print("{:20.15e}".format(m*exp))

1.401298464324817e-45
